In [3]:
class KnowledgeBase:
    def __init__(self):
        self.facts = {
            "American(Robert)": True,
            "Missile(T1)": True,
            "Owns(A, T1)": True,
            "Enemy(A, America)": True,
            "Sells(Robert, T1, A)": True
        }
        self.rules = [
            {"conditions": ["Missile(x)"], "result": "Weapon(x)"},
            {"conditions": ["Enemy(x, America)"], "result": "Hostile(x)"},
            {
                "conditions": ["American(p)", "Weapon(q)", "Sells(p, q, r)", "Hostile(r)"],
                "result": "Criminal(p)"
            }
        ]

class ForwardChaining:
    def __init__(self, kb):
        self.kb = kb
        self.derived_facts = set(kb.facts.keys())

    def infer(self):
        new_fact_found = True
        print("Starting forward chaining with initial facts:", self.derived_facts)

        while new_fact_found:
            new_fact_found = False
            current_facts = self.derived_facts.copy()

            for rule in self.kb.rules:
                conditions = rule["conditions"]
                result = rule["result"]
                print(f"Evaluating rule: {conditions} => {result}")
                substitutions = self.match_conditions(conditions, current_facts)
                if substitutions:
                    print(f"Conditions matched with substitutions: {substitutions}")
                    derived_fact = self.apply_substitution(result, substitutions)
                    print(f"New derived fact: {derived_fact}")

                    if derived_fact not in self.derived_facts:
                        print(f"Adding new fact: {derived_fact}")
                        self.derived_facts.add(derived_fact)
                        self.kb.facts[derived_fact] = True
                        new_fact_found = True

            print("Current derived facts:", self.derived_facts)

        return self.derived_facts

    def match_conditions(self, conditions, facts):
        substitutions = {}
        for condition in conditions:
            matched = False
            for fact in facts:
                substitution = self.unify(condition, fact)
                if substitution:
                    substitutions.update(substitution)
                    matched = True
                    break
            if not matched:
                return None
        return substitutions

    def unify(self, condition, fact):
        print(f"Trying to unify: {condition} with {fact}")
        if "(" in condition and "(" in fact:
            condition_pred, condition_args = self.parse_predicate(condition)
            fact_pred, fact_args = self.parse_predicate(fact)
            if condition_pred == fact_pred and len(condition_args) == len(fact_args):
                return {condition_args[i]: fact_args[i] for i in range(len(condition_args)) if condition_args[i].islower()}
        return None

    def parse_predicate(self, statement):
        pred, args = statement.split("(")
        args = args.strip(")").split(",")
        return pred.strip(), [arg.strip() for arg in args]

    def apply_substitution(self, statement, substitutions):
        pred, args = self.parse_predicate(statement)
        substituted_args = [substitutions.get(arg, arg) for arg in args]
        return f"{pred}({', '.join(substituted_args)})"


kb = KnowledgeBase()
fc = ForwardChaining(kb)
result = fc.infer()

if "Criminal(Robert)" in result:
    print("Query Proven: Criminal(Robert) is true.")
else:
    print("Query Not Proven: Criminal(Robert) is not true.")

print("\nAKASH K S")
print("USN: 1BM22CS028")


Starting forward chaining with initial facts: {'American(Robert)', 'Enemy(A, America)', 'Owns(A, T1)', 'Missile(T1)', 'Sells(Robert, T1, A)'}
Evaluating rule: ['Missile(x)'] => Weapon(x)
Trying to unify: Missile(x) with American(Robert)
Trying to unify: Missile(x) with Enemy(A, America)
Trying to unify: Missile(x) with Owns(A, T1)
Trying to unify: Missile(x) with Missile(T1)
Conditions matched with substitutions: {'x': 'T1'}
New derived fact: Weapon(T1)
Adding new fact: Weapon(T1)
Evaluating rule: ['Enemy(x, America)'] => Hostile(x)
Trying to unify: Enemy(x, America) with American(Robert)
Trying to unify: Enemy(x, America) with Enemy(A, America)
Conditions matched with substitutions: {'x': 'A'}
New derived fact: Hostile(A)
Adding new fact: Hostile(A)
Evaluating rule: ['American(p)', 'Weapon(q)', 'Sells(p, q, r)', 'Hostile(r)'] => Criminal(p)
Trying to unify: American(p) with American(Robert)
Trying to unify: Weapon(q) with American(Robert)
Trying to unify: Weapon(q) with Enemy(A, Ameri